In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


In [ ]:
data_dir = r"E:\age detection\UTKFace"  # Update this to your dataset path


In [2]:
def data_generator(file_list, batch_size=32, img_size=(128, 128)):
    while True:
        for i in range(0, len(file_list), batch_size):
            batch_files = file_list[i:i + batch_size]
            images, ages = [], []
            for file in batch_files:
                img_path = os.path.join(data_dir, file)
                age = int(file.split('_')[0])  # Extract age from filename
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size) / 255.0  # Resize and normalize
                images.append(img)
                ages.append(age)
            yield np.array(images, dtype='float32'), np.array(ages, dtype='float32')


In [ ]:
# Load and split data
files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')]

In [ ]:
train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output for age
])

# Compile the model with MAE as the evaluation metric
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [ ]:
batch_size = 32
train_gen = data_generator(train_files, batch_size=batch_size)
test_gen = data_generator(test_files, batch_size=batch_size)

# Steps per epoch
steps_per_epoch = len(train_files) // batch_size
validation_steps = len(test_files) // batch_size


In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_gen,
    validation_steps=validation_steps,
    epochs=30  # You can adjust epochs based on your performance
)

In [ ]:
# Save the model
model.save("age_detection_model.h5")
print("Training completed and model saved!")


In [ ]:
test_loss, test_mae = model.evaluate(test_gen, steps=validation_steps)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")


In [ ]:
import matplotlib.pyplot as plt

# Plotting training history (Loss and MAE)
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# MAE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()

plt.show()
